In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torchmetrics import Accuracy
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [2]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Dota2Dataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [4]:
df = pd.read_csv("data.csv")
X = df.drop(["radiant_win"], axis = 1)
y = df["radiant_win"]
X = torch.from_numpy(X.values).long()
y = torch.from_numpy(y.values).long()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
train_dataset = Dota2Dataset(X_train, y_train)
test_dataset = Dota2Dataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=1000000, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000000, shuffle=False)

C:\Users\nobod\AppData\Local\Temp\ipykernel_15744\1638339005.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.long)
C:\Users\nobod\AppData\Local\Temp\ipykernel_15744\1638339005.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.float32)


In [6]:
class Dota2Model(nn.Module):
    def __init__(self, num_heroes, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_heroes, embedding_dim=embedding_dim)
        
        input_dim = (embedding_dim * 2) + 1 

        self.layers = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, out_features=hidden_dim // 8),
            nn.ReLU(),
            nn.Linear(hidden_dim // 8, out_features=hidden_dim // 8),
            nn.ReLU(),
            nn.Linear(hidden_dim // 8, out_features=hidden_dim // 8),
            nn.Linear(hidden_dim // 8, out_features=hidden_dim // 8),
            nn.Linear(hidden_dim // 8, out_features=1)
        )

    def forward(self, match):
        avg_rank_tier = match[:, 0]
        radiant_teams = match[:, 1:6]
        dire_teams = match[:, 6:]
        
        radiant_embeds = self.embedding(radiant_teams)
        dire_embeds = self.embedding(dire_teams)
        
        radiant_embeds = radiant_embeds.mean(dim=1)
        dire_embeds = dire_embeds.mean(dim=1)
        
        combined_teams = torch.cat([avg_rank_tier.unsqueeze(1), radiant_embeds, dire_embeds], dim=1)
        
        return self.layers(combined_teams)

In [7]:
dota_2_model = Dota2Model(X.max().item()+1, 256, 256).to(device)

In [8]:
loss_function = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.Adam(params=dota_2_model.parameters(), lr=0.01)
accuracy_function = Accuracy(task="binary").to(device)

In [9]:
def training_step(model: nn.Module, data_loader: DataLoader, 
                  loss_function: torch.nn, optimizer: torch.optim, 
                  accuracy_function: Accuracy):
    model.train()
    for batch, (match, match_results) in enumerate(data_loader):
        match, match_results = match.to(device), match_results.to(device)
        y_logits = model(match).squeeze()
        y_preds = torch.round(torch.sigmoid(y_logits))
        
        match_results = match_results.float()

        loss = loss_function(y_logits, match_results)
        acc = accuracy_function(match_results, y_preds)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Training accuracy: {acc*100}%")

In [10]:
def testing_step(model: nn.Module, data_loader: DataLoader, 
                 accuracy_function: Accuracy):
    model.eval()
    with torch.inference_mode():
        for match, match_results in data_loader:
            match, match_results = match.to(device), match_results.to(device)
            y_logits = model(match).squeeze()
            y_preds = torch.round(torch.sigmoid(y_logits))
            
            match_results = match_results.float()

            acc = accuracy_function(match_results, y_preds)
        print(f"Testing accuracy: {acc*100}%")

In [11]:
epochs = 24
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    training_step(dota_2_model, train_loader, loss_function, optimizer, accuracy_function)
    testing_step(dota_2_model, test_loader, accuracy_function)

Epoch 1/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 2/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 3/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 4/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 5/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 6/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 7/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 8/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 9/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 10/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 11/24
Training accuracy: 58.25104522705078%
Testing accuracy: 58.24758529663086%
Epoch 12/24
Training accuracy: 58.25104522705078%
Te

In [12]:
torch.save(obj=dota_2_model.state_dict(),
           f="model.pth")